In [1]:
!pip install autogluon

In [2]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) 

In [3]:
train = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/train.csv")
test = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/test.csv")
sample_submission= pd.read_csv("C:/Users/user/Desktop/흡연여부/open/sample_submission.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7000 non-null   object 
 1   나이        7000 non-null   int64  
 2   키(cm)     7000 non-null   int64  
 3   몸무게(kg)   7000 non-null   int64  
 4   BMI       7000 non-null   float64
 5   시력        7000 non-null   float64
 6   충치        7000 non-null   int64  
 7   공복 혈당     7000 non-null   int64  
 8   혈압        7000 non-null   int64  
 9   중성 지방     7000 non-null   int64  
 10  혈청 크레아티닌  7000 non-null   float64
 11  콜레스테롤     7000 non-null   int64  
 12  고밀도지단백    7000 non-null   int64  
 13  저밀도지단백    7000 non-null   int64  
 14  헤모글로빈     7000 non-null   float64
 15  요 단백      7000 non-null   int64  
 16  간 효소율     7000 non-null   float64
 17  label     7000 non-null   int64  
dtypes: float64(5), int64(12), object(1)
memory usage: 984.5+ KB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        3000 non-null   object 
 1   나이        3000 non-null   int64  
 2   키(cm)     3000 non-null   int64  
 3   몸무게(kg)   3000 non-null   int64  
 4   BMI       3000 non-null   float64
 5   시력        3000 non-null   float64
 6   충치        3000 non-null   int64  
 7   공복 혈당     3000 non-null   int64  
 8   혈압        3000 non-null   int64  
 9   중성 지방     3000 non-null   int64  
 10  혈청 크레아티닌  3000 non-null   float64
 11  콜레스테롤     3000 non-null   int64  
 12  고밀도지단백    3000 non-null   int64  
 13  저밀도지단백    3000 non-null   int64  
 14  헤모글로빈     3000 non-null   float64
 15  요 단백      3000 non-null   int64  
 16  간 효소율     3000 non-null   float64
dtypes: float64(5), int64(11), object(1)
memory usage: 398.6+ KB


In [6]:
train = train.drop(['ID'], axis = 1)
test = test.drop('ID', axis = 1)

In [7]:
train['혈청 크레아티닌'].describe()

count    7000.000000
mean        0.884900
std         0.241523
min         0.100000
25%         0.800000
50%         0.900000
75%         1.000000
max        10.000000
Name: 혈청 크레아티닌, dtype: float64

In [8]:
# 파생변수 =>혈청 크레아티닌값에 따라 정상/비정상 여부를 판단하여 1 또는 0으로 기록
train['혈청 크레아티닌_status'] = train['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)

In [10]:
## 상관분석

import scipy.stats as stats

PSI_corr = pd.DataFrame()
PSI_corr

Y = train['label'].values

for item in train.drop(columns=['label']).columns :
    X = train[item].values
    new_data = {
        'variable' : [item],
        'Correlation' : [round(stats.pearsonr(X, Y)[0], 2)],
        'p-value' : [round(stats.pearsonr(X, Y)[1], 3)]
    }
   
    new_df = pd.DataFrame(new_data)
    PSI_corr = pd.concat([PSI_corr, new_df])

PSI_corr = PSI_corr.reset_index(drop=True)
PSI_corr

,variable,Correlation,p-value
0,나이,-0.17,0.000
1,키(cm),0.39,0.000
2,몸무게(kg),0.31,0.000
3,BMI,0.13,0.000
4,시력,0.07,0.000
5,충치,0.10,0.000
6,공복 혈당,0.11,0.000
7,혈압,0.02,0.110
8,중성 지방,0.25,0.000
9,혈청 크레아티닌,0.20,0.000


In [11]:
j = len(PSI_corr)
PSI_corr = PSI_corr.reset_index(drop=True)

for i in range(0, j) :
    if PSI_corr['p-value'][i] >= 0.05 :
        PSI_corr = PSI_corr.drop(i)
#     else :
#         if PSI_corr['Correlation'][i] > -0.35 and PSI_corr['Correlation'][i] < 0.35 :
#             PSI_corr = PSI_corr.drop(i)

PSI_corr = PSI_corr.sort_values('Correlation', ascending = False)
PSI_corr = PSI_corr.reset_index(drop=True)
PSI_corr

,variable,Correlation,p-value
0,헤모글로빈,0.40,0.000
1,키(cm),0.39,0.000
2,몸무게(kg),0.31,0.000
3,중성 지방,0.25,0.000
4,혈청 크레아티닌_status,0.25,0.000
5,혈청 크레아티닌,0.20,0.000
6,BMI,0.13,0.000
7,공복 혈당,0.11,0.000
8,충치,0.10,0.000
9,시력,0.07,0.000


In [12]:
#train 설정- 상관분석을 고려한 유의하지 않은 변수 제거
train = train.drop(['콜레스테롤', '요 단백'], axis=1)


In [13]:
# train데이터에 했던 전처리 과정 테스트데이터에도 진행
test['혈청 크레아티닌_status'] = test['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)
test = test.drop(['콜레스테롤', '요 단백'], axis=1)

In [14]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

label = 'label'
eval_metric = 'accuracy'
time_limit = 1200

In [15]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data,  presets='best_quality', time_limit=time_limit, 
      num_stack_levels=5)

No path specified. Models will be saved in: "AutogluonModels\ag-20230906_051424\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=5, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "AutogluonModels\ag-20230906_051424\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   24.85 GB / 237.11 GB (10.5%)
Train Data Rows:    7000
Train Data Columns: 15
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 =

In [16]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L7,0.781143,29.901652,817.452775,0.030900,3.737194,7,True,66
1,LightGBM_BAG_L6,0.781000,29.782729,801.961628,0.062470,6.304742,6,True,62
2,LightGBMXT_BAG_L6,0.778286,29.860267,802.991244,0.140008,7.334357,6,True,61
3,CatBoost_BAG_L6,0.777857,29.808282,807.410839,0.088023,11.753952,6,True,65
4,RandomForestEntr_BAG_L6,0.776714,30.502851,801.926477,0.782593,6.269590,6,True,64
...,...,...,...,...,...,...,...,...,...
61,RandomForestGini_BAG_L1,0.736000,0.950083,3.534706,0.950083,3.534706,1,True,5
62,ExtraTreesGini_BAG_L1,0.725571,1.000708,2.498837,1.000708,2.498837,1,True,8
63,ExtraTreesEntr_BAG_L1,0.725143,1.093384,2.461494,1.093384,2.461494,1,True,9
64,KNeighborsDist_BAG_L1,0.684000,0.516157,0.045773,0.516157,0.045773,1,True,2


In [17]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

In [18]:
sample_submission['label'] = model_pred
sample_submission

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,0


In [20]:
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/submission20minute.csv",index=False)